In [26]:
mainDir = '/Users/vijay/CarND-Behavioral-Cloning-P3/data/'

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import cv2
import numpy as np

df = pd.read_csv(mainDir + 'driving_log.csv', sep=',',header=0)
#df = df[df['steering'] > 0.05 or df['steering'] < -0.05]

X = df["center"]
X = X.map(lambda img : np.reshape(cv2.imread(mainDir + img, 0), (160, 320, 1)))
Y = df["steering"]

X = np.asarray(X.tolist())
Y = np.asarray(Y.tolist())

## shuffle
X, Y = shuffle(X, Y, random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.0, random_state=42)

In [19]:
x_new = [val > 0.05 for val in Y]
print(len(Y))
print(len(x_new))
print(type(Y))

8036
8036
<class 'numpy.ndarray'>


In [27]:
def normalize_grayscale(image_data):
    a = -0.5
    b = 0.5
    grayscale_min = 0
    grayscale_max = 255
    return a + ( ( (image_data - grayscale_min)*(b - a) )/( grayscale_max - grayscale_min ) )

## normalize the images
X_train = normalize_grayscale(X_train)
X_val = normalize_grayscale(X_train)

In [88]:
print(type(X_train))
print(X_train[0:2].shape)
print(X_train.shape)
print(X_train[0].shape)

stuff = np.asarray(X_train.tolist())
print(type(stuff))
print(stuff.shape)

#for elem in stuff:

# X_train2 = np.ndarray(shape=(None, 160, 320, 1), dtype=int)
# for img in X_train[0:10]:
#     np.append(X_train2, [img], axis=0)

# #X_train2 = X_train.as_matrix()
# print(type(X_train2))
# print(X_train2.shape)
# print(X_train2)

<class 'pandas.core.series.Series'>
(2,)
(1558,)
(160, 320)
<class 'numpy.ndarray'>
(1558, 160, 320)


In [100]:
import scipy.stats as stats
import pylab as pl

np.bincount(y_train) 

TypeError: Cannot cast array data from dtype('float64') to dtype('int64') according to the rule 'safe'

In [29]:
from keras.models import Sequential
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D

model = Sequential()
model.add(Convolution2D(24, 5, 5, input_shape=(160, 320, 1), subsample=(2, 2), activation='relu'))
model.add(Convolution2D(36, 5, 5, subsample=(2, 2), activation='relu'))
model.add(Convolution2D(48, 5, 5, subsample=(2, 2), activation='relu'))
model.add(Convolution2D(64, 3, 3, subsample=(1, 1), activation='relu'))
model.add(Convolution2D(64, 3, 3, subsample=(1, 1), activation='relu'))
model.add(Flatten())
model.add(Dense(1164, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))

model.compile('adam', 'mean_squared_error', ['accuracy'])
history = model.fit(X_train, y_train, nb_epoch=5, batch_size=128, validation_split=0.2)

Train on 6428 samples, validate on 1608 samples
Epoch 1/5
6428/6428 [==============================] - 437s - loss: 0.0158 - acc: 0.5409 - val_loss: 0.0134 - val_acc: 0.5498
Epoch 2/5
6428/6428 [==============================] - 390s - loss: 0.0125 - acc: 0.5409 - val_loss: 0.0118 - val_acc: 0.5498
Epoch 3/5
6428/6428 [==============================] - 379s - loss: 0.0109 - acc: 0.5409 - val_loss: 0.0092 - val_acc: 0.5498
Epoch 4/5
6428/6428 [==============================] - 391s - loss: 0.0092 - acc: 0.5409 - val_loss: 0.0090 - val_acc: 0.5498
Epoch 5/5
6428/6428 [==============================] - 317s - loss: 0.0087 - acc: 0.5409 - val_loss: 0.0084 - val_acc: 0.5498


In [30]:
from keras.models import load_model
model.save('model.h5')  # creates a HDF5 file 'my_model.h5'